# Position Holding Strategy (2-6 Months)

**Goal**: Hold quality stocks for 2-6 months to capture 20-40%+ trends

**Differences from Swing Trading (02.1)**:
- Entry: Breakouts instead of pullbacks
- Exit: Trailing stops instead of fixed targets
- Hold time: 2-6 months vs 2 weeks
- Trade frequency: 5-15/year vs 30-50/year
- Position sizing: Fewer, larger positions

**Strategy Framework**:
- BUY: Quality stocks breaking to new highs with strong momentum
- HOLD: Ride the trend with trailing stop protection
- SELL: When trend breaks or trailing stop hits

In [ ]:
# CELL 1: Setup
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from src.data_fetcher import DataFetcher
from src.indicators import sma, ema, rsi, macd, bollinger_bands, atr, adx
from src.market import MarketRegime, check_market_health

# Initialize
fetcher = DataFetcher()
print("Position Holding Strategy - Ready")
print("="*60)

## Step 1: Check Market Regime

Only look for positions during bull markets.

In [ ]:
# CELL 2: Check Market Health
market_regime = check_market_health(verbose=True, check_breadth=False)

if market_regime['healthy']:
    print("\nGREEN LIGHT: Market is healthy for position entries")
else:
    print("\nRED LIGHT: Wait for market to stabilize before entering positions")

## Step 2: Define Stock Universe

Focus on quality growth stocks with strong fundamentals.

In [ ]:
# CELL 3: Stock Universe
# Quality growth stocks across sectors
stock_universe = [
    # Tech Leaders
    'NVDA', 'AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'TSLA',
    # Semiconductors
    'AMD', 'AVGO', 'TSM', 'ASML',
    # Software/Cloud
    'CRM', 'ADBE', 'NOW', 'SNOW', 'PLTR',
    # Fintech/Payments
    'SQ', 'PYPL', 'COIN', 'HOOD', 'SOFI',
    # EV/Clean Energy
    'RIVN', 'LCID', 'ENPH',
    # Growth Stocks
    'SHOP', 'ROKU', 'NET', 'DKNG', 'ABNB'
]

print(f"Analyzing {len(stock_universe)} stocks for position entries")
print(f"Period: Last 1 year of data")

## Step 3: Calculate Fundamental Quality Score

Same scoring system as swing trading - focus on financial health.

In [ ]:
# CELL 4: Fundamental Scoring Function
def calculate_fundamental_score(ticker):
    """
    Score stock fundamentals (0-100)
    Higher threshold for position holding (75+ required)
    """
    try:
        import yfinance as yf
        stock = yf.Ticker(ticker)
        info = stock.info
        
        score = 0
        max_score = 100
        details = {}
        
        # 1. Profitability (30 points)
        profit_margin = info.get('profitMargin', 0)
        if profit_margin:
            if profit_margin > 0.20:
                score += 30
                details['profit_margin'] = 'Excellent (>20%)'
            elif profit_margin > 0.10:
                score += 20
                details['profit_margin'] = 'Good (10-20%)'
            elif profit_margin > 0:
                score += 10
                details['profit_margin'] = 'Profitable'
            else:
                details['profit_margin'] = 'Unprofitable'
        
        # 2. Revenue Growth (25 points)
        revenue_growth = info.get('revenueGrowth', 0)
        if revenue_growth:
            if revenue_growth > 0.30:
                score += 25
                details['revenue_growth'] = f'Excellent ({revenue_growth*100:.0f}%)'
            elif revenue_growth > 0.15:
                score += 18
                details['revenue_growth'] = f'Good ({revenue_growth*100:.0f}%)'
            elif revenue_growth > 0:
                score += 10
                details['revenue_growth'] = f'Positive ({revenue_growth*100:.0f}%)'
            else:
                details['revenue_growth'] = 'Declining'
        
        # 3. Financial Health (20 points)
        debt_to_equity = info.get('debtToEquity')
        if debt_to_equity is not None:
            if debt_to_equity < 50:
                score += 20
                details['debt'] = 'Low debt'
            elif debt_to_equity < 100:
                score += 12
                details['debt'] = 'Moderate debt'
            else:
                score += 5
                details['debt'] = 'High debt'
        
        # 4. Market Cap (15 points) - prefer larger companies for positions
        market_cap = info.get('marketCap', 0)
        if market_cap > 50e9:  # >$50B
            score += 15
            details['size'] = 'Large cap'
        elif market_cap > 10e9:  # >$10B
            score += 12
            details['size'] = 'Mid cap'
        else:
            score += 8
            details['size'] = 'Small cap'
        
        # 5. Institutional Ownership (10 points) - smart money
        inst_ownership = info.get('heldPercentInstitutions', 0)
        if inst_ownership > 0.60:
            score += 10
            details['institutions'] = f'High ({inst_ownership*100:.0f}%)'
        elif inst_ownership > 0.40:
            score += 6
            details['institutions'] = f'Moderate ({inst_ownership*100:.0f}%)'
        else:
            score += 3
            details['institutions'] = f'Low ({inst_ownership*100:.0f}%)'
        
        return {'score': score, 'details': details}
    
    except Exception as e:
        return {'score': 0, 'details': {'error': str(e)}}

print("Fundamental scoring function loaded")
print("Threshold: 75+ for position entries (elite stocks only)")

## Step 4: Position Entry Scanner

Find stocks breaking to new highs with strong momentum.

In [ ]:
# CELL 5: Position Entry Logic
def scan_for_positions(tickers, market_regime):
    """
    Scan for position entry opportunities
    
    Entry Criteria:
    1. Market regime = BULL
    2. Fundamental score >= 75 (elite only)
    3. Price at 3-month high (strong trend)
    4. ADX > 25 (strong momentum)
    5. RSI 50-70 (healthy, not overbought)
    6. Volume > 20-day average (institutional interest)
    """
    
    # Check market health first
    if not market_regime or not market_regime.get('healthy', False):
        print("Market is unhealthy - no entries recommended")
        return []
    
    opportunities = []
    
    for ticker in tickers:
        try:
            # Fetch data
            df = fetcher.fetch(ticker, period='1y')
            if df is None or len(df) < 200:
                continue
            
            # Calculate indicators
            df['SMA_20'] = sma(df['Close'], 20)
            df['SMA_50'] = sma(df['Close'], 50)
            df['SMA_200'] = sma(df['Close'], 200)
            df['RSI'] = rsi(df['Close'], 14)
            df['ATR'] = atr(df['High'], df['Low'], df['Close'], 14)
            
            adx_data = adx(df['High'], df['Low'], df['Close'], 14)
            df['ADX'] = adx_data['ADX']
            
            df['Volume_MA20'] = df['Volume'].rolling(20).mean()
            df = df.dropna()
            
            if len(df) < 60:
                continue
            
            # Current values
            current = df.iloc[-1]
            current_price = current['Close']
            
            # Calculate 3-month high
            high_3m = df['High'].tail(60).max()
            
            # Technical checks
            at_3m_high = current_price >= high_3m * 0.98  # Within 2% of 3-month high
            strong_momentum = current['ADX'] > 25
            healthy_rsi = 50 < current['RSI'] < 70
            above_200ma = current_price > current['SMA_200']
            volume_surge = current['Volume'] > current['Volume_MA20'] * 1.2
            uptrend = current['SMA_20'] > current['SMA_50'] > current['SMA_200']
            
            # Technical score
            tech_checks = [
                at_3m_high,
                strong_momentum,
                healthy_rsi,
                above_200ma,
                volume_surge,
                uptrend
            ]
            tech_score = sum(tech_checks) / len(tech_checks) * 100
            
            # Fundamental score
            fund_data = calculate_fundamental_score(ticker)
            fund_score = fund_data['score']
            
            # Entry signal: All criteria must pass
            is_entry = (
                fund_score >= 75 and  # Elite fundamentals only
                at_3m_high and        # Breaking out
                strong_momentum and   # Strong trend
                healthy_rsi and       # Not overbought
                above_200ma           # Long-term uptrend
            )
            
            if is_entry or fund_score >= 70:  # Show near-misses too
                opportunities.append({
                    'ticker': ticker,
                    'price': current_price,
                    'signal': 'BUY NOW' if is_entry else 'WATCH',
                    'fund_score': fund_score,
                    'tech_score': tech_score,
                    'rsi': current['RSI'],
                    'adx': current['ADX'],
                    'atr': current['ATR'],
                    'volume_ratio': current['Volume'] / current['Volume_MA20'],
                    'distance_to_3m_high': (current_price / high_3m - 1) * 100,
                    'stop_loss': current_price - (2 * current['ATR']),
                    'initial_risk_pct': (2 * current['ATR'] / current_price) * 100
                })
        
        except Exception as e:
            continue
    
    return opportunities

print("Position scanner loaded")
print("Scanning for breakout + momentum entries...")

In [ ]:
# CELL 6: Run Scanner
print("Scanning stock universe for position entries...\n")

opportunities = scan_for_positions(stock_universe, market_regime)

if opportunities:
    df_opps = pd.DataFrame(opportunities)
    df_opps = df_opps.sort_values(['signal', 'fund_score'], ascending=[True, False])
    
    # Display results
    print("="*100)
    print("POSITION ENTRY OPPORTUNITIES")
    print("="*100)
    
    for _, row in df_opps.iterrows():
        print(f"\n{row['ticker']} - {row['signal']}")
        print(f"  Price: ${row['price']:.2f}")
        print(f"  Quality: {row['fund_score']}/100 (Fund) | {row['tech_score']:.0f}/100 (Tech)")
        print(f"  Momentum: RSI {row['rsi']:.1f} | ADX {row['adx']:.1f}")
        print(f"  Volume: {row['volume_ratio']:.1f}x average")
        print(f"  Entry: ${row['price']:.2f}")
        print(f"  Stop Loss: ${row['stop_loss']:.2f} (-{row['initial_risk_pct']:.1f}%)")
        print(f"  Position Risk: {row['initial_risk_pct']:.1f}% per trade")
    
    print("\n" + "="*100)
    print(f"Total: {len(df_opps[df_opps['signal']=='BUY NOW'])} BUY signals | {len(df_opps[df_opps['signal']=='WATCH'])} WATCH list")
    print("="*100)
else:
    print("No opportunities found. Wait for better setups.")

## Step 5: Position Management & Exit Rules

How to manage and exit positions for maximum profit.

In [ ]:
# CELL 7: Exit Strategy
print("="*80)
print("POSITION EXIT RULES")
print("="*80)

print("""
Exit immediately if ANY of these conditions are met:

1. TRAILING STOP (Primary exit - let winners run)
   - Trail by 15% from highest peak
   - Example: Stock goes from $100 to $150 (50% gain)
     → Stop is now at $127.50 ($150 * 0.85)
   - Protects profits while allowing trend to continue

2. TREND BREAKDOWN (Technical deterioration)
   - Close below 50-day MA AND RSI < 40
   - Signals trend reversal

3. MARKET REGIME SHIFT (Protect capital)
   - Market regime changes to CORRECTION or BEAR
   - Exit all positions to preserve capital

4. TIME EXIT (Prevent dead money)
   - 6 months maximum hold time
   - If not working after 6 months, redeploy capital

5. FUNDAMENTAL DETERIORATION (Rare)
   - Major negative news (earnings miss, regulatory issues)
   - Management changes
   - Business model threats

DAILY CHECKLIST:
□ Check trailing stop levels
□ Verify market regime is still BULL
□ Monitor for trend breakdown signals
□ Track days in position (toward 6-month max)

WEEKLY CHECKLIST:
□ Review fundamentals (earnings, news)
□ Recalculate position sizes if needed
□ Update trailing stops
□ Look for new position opportunities

COMMON MISTAKES TO AVOID:
❌ Moving stops further away (always trail tighter)
❌ Hoping losing positions recover (respect stops)
❌ Taking profits too early (let trailing stop do its job)
❌ Adding to losing positions (only add to winners)
""")

print("="*80)

## Step 6: Backtest the Strategy

Test on historical data to validate performance.

In [ ]:
# CELL 8: Backtest Function
def backtest_position_strategy(ticker, period='1y'):
    """
    Backtest position holding strategy
    
    Entry: 3-month high + ADX>25 + RSI 50-70 + above 200 MA
    Exit: 15% trailing stop OR trend breakdown OR 6 months
    """
    
    # Fetch data
    df = fetcher.fetch(ticker, period='2y')  # Need 2y for lookback
    if df is None or len(df) < 250:
        return None
    
    # Calculate indicators
    df['SMA_20'] = sma(df['Close'], 20)
    df['SMA_50'] = sma(df['Close'], 50)
    df['SMA_200'] = sma(df['Close'], 200)
    df['RSI'] = rsi(df['Close'], 14)
    df['ATR'] = atr(df['High'], df['Low'], df['Close'], 14)
    
    adx_data = adx(df['High'], df['Low'], df['Close'], 14)
    df['ADX'] = adx_data['ADX']
    
    df = df.dropna()
    
    # Fetch SPY for regime detection
    spy_df = fetcher.fetch('SPY', period='2y')
    if spy_df is None:
        return None
    
    spy_df['SMA_50'] = sma(spy_df['Close'], 50)
    spy_df['SMA_200'] = sma(spy_df['Close'], 200)
    spy_df = spy_df.dropna()
    
    # Backtest last 1 year
    cutoff_date = df.index[-1] - timedelta(days=365)
    df_test = df[df.index >= cutoff_date].copy()
    
    trades = []
    in_position = False
    entry_price = 0
    entry_date = None
    highest_price = 0
    trailing_stop = 0
    
    for i in range(60, len(df_test)):  # Need 60 days for 3-month high
        current = df_test.iloc[i]
        current_date = current.name
        
        # Check market regime
        spy_at_date = spy_df[spy_df.index <= current_date]
        if len(spy_at_date) < 200:
            market_is_bull = True
        else:
            spy_current = spy_at_date.iloc[-1]
            market_is_bull = (spy_current['Close'] > spy_current['SMA_50'] and 
                            spy_current['Close'] > spy_current['SMA_200'])
        
        if not in_position:
            # Skip if market is not bull
            if not market_is_bull:
                continue
            
            # Calculate 3-month high from current position looking back
            lookback_data = df_test.iloc[max(0, i-60):i]
            high_3m = lookback_data['High'].max()
            
            # Entry conditions
            at_3m_high = current['Close'] >= high_3m * 0.98
            strong_momentum = current['ADX'] > 25
            healthy_rsi = 50 < current['RSI'] < 70
            above_200ma = current['Close'] > current['SMA_200']
            
            if at_3m_high and strong_momentum and healthy_rsi and above_200ma:
                in_position = True
                entry_price = current['Close']
                entry_date = current.name
                highest_price = entry_price
                trailing_stop = entry_price * 0.85  # 15% trailing stop
        
        else:
            # Update highest price and trailing stop
            if current['Close'] > highest_price:
                highest_price = current['Close']
                trailing_stop = highest_price * 0.85  # Trail by 15%
            
            days_in_trade = (current.name - entry_date).days
            
            # Exit conditions
            hit_trailing_stop = current['Close'] <= trailing_stop
            trend_breakdown = current['Close'] < current['SMA_50'] and current['RSI'] < 40
            market_regime_shift = not market_is_bull
            time_exit = days_in_trade > 180  # 6 months
            
            if hit_trailing_stop:
                exit_price = trailing_stop
                exit_reason = 'Trailing Stop'
            elif trend_breakdown:
                exit_price = current['Close']
                exit_reason = 'Trend Breakdown'
            elif market_regime_shift:
                exit_price = current['Close']
                exit_reason = 'Market Regime'
            elif time_exit:
                exit_price = current['Close']
                exit_reason = 'Time Exit'
            else:
                continue
            
            # Record trade
            pnl_pct = ((exit_price - entry_price) / entry_price) * 100
            max_gain_pct = ((highest_price - entry_price) / entry_price) * 100
            
            trades.append({
                'entry_date': entry_date,
                'exit_date': current.name,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'highest_price': highest_price,
                'pnl_pct': pnl_pct,
                'max_gain_pct': max_gain_pct,
                'exit_reason': exit_reason,
                'days_held': days_in_trade
            })
            
            in_position = False
    
    return pd.DataFrame(trades) if trades else None

print("Backtesting function loaded")

In [ ]:
# CELL 9: Run Backtest
print("="*80)
print("BACKTESTING POSITION HOLDING STRATEGY")
print("="*80)

# Test on top stocks
backtest_tickers = ['NVDA', 'AAPL', 'MSFT', 'META', 'PLTR', 'SOFI', 'COIN', 'TSM']
print(f"\nTesting on: {', '.join(backtest_tickers)}")
print(f"Period: Last 1 year")
print(f"Strategy: Breakout + 15% trailing stop\n")

all_results = {}
for ticker in backtest_tickers:
    print(f"Backtesting {ticker}...", end=" ")
    result = backtest_position_strategy(ticker)
    if result is not None and not result.empty:
        all_results[ticker] = result
        print(f"{len(result)} trades")
    else:
        print("No trades")

# Combine all trades
if all_results:
    all_trades = pd.concat(all_results.values(), keys=all_results.keys(), names=['ticker', 'trade_id'])
    
    # Calculate metrics
    total_trades = len(all_trades)
    winners = all_trades[all_trades['pnl_pct'] > 0]
    losers = all_trades[all_trades['pnl_pct'] <= 0]
    
    win_rate = len(winners) / total_trades * 100
    avg_win = winners['pnl_pct'].mean() if len(winners) > 0 else 0
    avg_loss = losers['pnl_pct'].mean() if len(losers) > 0 else 0
    total_return = all_trades['pnl_pct'].sum()
    avg_return = all_trades['pnl_pct'].mean()
    avg_hold = all_trades['days_held'].mean()
    
    profit_factor = abs(winners['pnl_pct'].sum() / losers['pnl_pct'].sum()) if len(losers) > 0 else float('inf')
    
    print("\n" + "="*80)
    print("BACKTEST RESULTS")
    print("="*80)
    print(f"\nTotal Trades: {total_trades}")
    print(f"Winners: {len(winners)} ({win_rate:.1f}%)")
    print(f"Losers: {len(losers)} ({100-win_rate:.1f}%)")
    print(f"\nAverage Win: +{avg_win:.2f}%")
    print(f"Average Loss: {avg_loss:.2f}%")
    print(f"Profit Factor: {profit_factor:.2f}x")
    print(f"\nTotal Return: {total_return:.2f}%")
    print(f"Average Return: {avg_return:.2f}% per trade")
    print(f"Average Hold: {avg_hold:.1f} days ({avg_hold/30:.1f} months)")
    
    # Exit reason breakdown
    print(f"\nExit Reasons:")
    exit_counts = all_trades['exit_reason'].value_counts()
    for reason, count in exit_counts.items():
        pct = count / total_trades * 100
        print(f"  {reason}: {count} ({pct:.1f}%)")
    
    # Per-stock performance
    print(f"\nPer-Stock Performance:")
    for ticker in backtest_tickers:
        if ticker in all_results:
            stock_trades = all_results[ticker]
            stock_return = stock_trades['pnl_pct'].sum()
            stock_wr = len(stock_trades[stock_trades['pnl_pct'] > 0]) / len(stock_trades) * 100
            print(f"  {ticker}: {stock_return:+.2f}% | {stock_wr:.0f}% win rate | {len(stock_trades)} trades")
    
    # Average max gain captured
    avg_max_gain = all_trades['max_gain_pct'].mean()
    capture_rate = (avg_return / avg_max_gain * 100) if avg_max_gain > 0 else 0
    print(f"\nTrailing Stop Efficiency:")
    print(f"  Average peak gain: +{avg_max_gain:.2f}%")
    print(f"  Average realized gain: +{avg_return:.2f}%")
    print(f"  Capture rate: {capture_rate:.1f}% (how much of peak gains captured)")
    
    print("\n" + "="*80)
    
    if profit_factor > 2.0 and win_rate > 50:
        print("STATUS: EXCELLENT STRATEGY")
        print("Ready for paper trading")
    elif profit_factor > 1.5:
        print("STATUS: GOOD STRATEGY")
        print("Consider refinements before live trading")
    else:
        print("STATUS: NEEDS IMPROVEMENT")
        print("Refine entry/exit rules")
    
    print("="*80)
else:
    print("\nNo trades generated in backtest period")

## Step 7: Compare to Swing Trading

Side-by-side comparison with the swing trading strategy from 02.1.

In [ ]:
# CELL 10: Strategy Comparison
print("="*80)
print("STRATEGY COMPARISON: Swing Trading vs Position Holding")
print("="*80)

comparison = {
    'Metric': [
        'Entry Style',
        'Typical Hold Time',
        'Trades per Year',
        'Average Gain/Trade',
        'Time Commitment',
        'Stress Level',
        'Capital Efficiency',
        'Best For'
    ],
    'Swing Trading (02.1)': [
        'Pullbacks (oversold bounces)',
        '12 days (2 weeks)',
        '30-50 trades',
        '5-10%',
        'Daily monitoring',
        'Medium',
        'High (quick turnover)',
        'Active traders, quick gains'
    ],
    'Position Holding (02.3)': [
        'Breakouts (momentum)',
        '60-90 days (2-3 months)',
        '5-15 trades',
        '15-30%',
        'Daily check, weekly review',
        'Low',
        'Lower (patient capital)',
        'Busy professionals, trend riders'
    ]
}

df_comparison = pd.DataFrame(comparison)
print("\n", df_comparison.to_string(index=False))

print("\n" + "="*80)
print("RECOMMENDATION")
print("="*80)
print("""
Use BOTH strategies:

1. Position Holding (70% of capital)
   - 3-5 positions maximum
   - Let big winners run
   - Less time intensive
   - Lower stress

2. Swing Trading (30% of capital)
   - Capitalize on short-term bounces
   - Keep skills sharp
   - Quick profits during consolidation
   - More activity

This combination:
- Balances activity vs patience
- Captures both quick bounces AND big trends
- Reduces stress (most capital in positions)
- Maintains market engagement
""")
print("="*80)

## Summary

**Position Holding Strategy** is designed for traders who want:
- Larger gains per trade (20-40%+)
- Fewer trades (5-15 per year)
- Less time monitoring
- Ability to ride major trends

**Next Steps:**
1. Run the scanner (Cell 6) to find current opportunities
2. Review backtest results (Cell 9) to validate strategy
3. Paper trade 2-3 positions for 3 months
4. If successful, allocate 70% of capital to position holding
5. Keep 30% for swing trading (02.1) for more active trading